In [2]:
!pip install -U torchtext==0.8.0
from torchtext.data import Field, BucketIterator, TabularDataset




     |████████████████████████████████| 6.9 MB 9.4 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0


OSError: ignored

In [3]:
from torchtext.data import field, BucketIterator, TabularDataset
!pip install torchtext --upgrade

     |████████████████████████████████| 7.6 MB 9.8 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.8.0
    Uninstalling torchtext-0.8.0:
      Successfully uninstalled torchtext-0.8.0


In [4]:
import pandas as pd
import csv 
import os 
import pickle
import numpy as np
import torch 
import torchtext
import torch.nn.functional as F

from torchtext.data import get_tokenizer
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import ngrams_iterator

import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import itertools
import string


In [5]:
from torchtext.data.metrics import bleu_score

import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')

In [6]:
pth_1 =  "/content/drive/MyDrive/Dataset/Dakshina/hi/hi.translit.sampled.train.tsv"


In [7]:
#with open(pth_1,'r') as tsv:
#  file = csv.reader(tsv,delimiter = "\t")
df = pd.read_csv(pth_1,sep="\t",names=["Hindi","roman","freq"]) 
df = df[['Hindi','roman']]

In [8]:
from sklearn.model_selection import train_test_split
train , val = train_test_split(df,test_size=0.1)
train.to_csv("train.csv",index=False)
val.to_csv("val.csv",index=False)

,Hindi,roman
32333.0,मौन,moun
3856.0,इकलौती,eklauti
9248.0,क्वायर,quar
32826.0,रंगे,range
15950.0,डायमंड,diamond
...,...,...
11547.0,चतुर्भुज,chaturbhurj
8388.0,कूर्म,koorma
21573.0,पटकथा,patkatha
14183.0,जिम्मेदार,jimmedar


In [ ]:
#with open(pth_1,'r') as tsv:


,Hindi,roman
0,मौन,moun
1,इकलौती,eklauti
2,क्वायर,quar
3,रंगे,range
4,डायमंड,diamond
...,...,...
438,चतुर्भुज,chaturbhurj
439,कूर्म,koorma
440,पटकथा,patkatha
441,जिम्मेदार,jimmedar


In [8]:
hindi = df["Hindi"]
roman = df["roman"]

In [ ]:
max([len(i) for i in hindi]), max([len(str(i)) for i in roman])

(19, 20)

In [ ]:
i = next(val_dataloader)
print(i.roman)

TypeError: ignored

In [ ]:
tokenize_("चतुर्भुज")

['च',
 'त',
 'ु',
 'र',
 '्',
 'भ',
 'ु',
 'ज',
 'च त',
 'त ु',
 'ु र',
 'र ्',
 '् भ',
 'भ ु',
 'ु ज']

In [9]:
def tokenize_hi(words):
    n_grams = 2
    chars= [tok for tok in words]
    chars =list(ngrams_iterator(chars,n_grams))
    return chars
def tokenize_(words):
  return [tok for tok in words]
hi_text = field.Field(tokenize=tokenize_hi,use_vocab=True)
en_text = field.Field(tokenize=tokenize_, init_token = "<sos>", use_vocab=True,eos_token = "<eos>",is_target=True)

In [10]:
data_fields = [('Hindi', hi_text), ('roman', en_text)]
train= TabularDataset(path = "/content/train.csv",
                            format='csv', skip_header=True,fields=data_fields)
val= TabularDataset(path = "/content/val.csv",
                            format='csv', skip_header=True,fields=data_fields)

hi_text.build_vocab(train.Hindi,val.Hindi)
en_text.build_vocab(train.roman,val.roman)

In [11]:
hi_text.vocab

In [12]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
device

'cuda'

In [13]:
train_dataloader,val_dataloader = torchtext.data.BucketIterator.splits(
    
                              # Datasets for iterator to draw data from
                              (train,val),

                              # Tuple of train and validation batch sizes.
                              batch_sizes=(512,32),

                              # Device to load batches on.
                              device=device, 

                              # Function to use for sorting examples.
                              sort_key=lambda x: len(x.Hindi),


                              # Repeat the iterator for multiple epochs.
                              repeat=False, 

                              # Sort all examples in data using `sort_key`.
                              sort=False, 

                              # Shuffle data on each epoch run.
                              shuffle=True,

                              # Use `sort_key` to sort examples in each batch.
                              sort_within_batch = False,
                              
                              )

In [27]:
td,vd = torchtext.data.BucketIterator.splits(
    
                              # Datasets for iterator to draw data from
                              (train,val),

                              # Tuple of train and validation batch sizes.
                              batch_sizes=(32,32),

                              # Device to load batches on.
                              device='cpu', 

                              # Function to use for sorting examples.
                              sort_key=lambda x: len(x.Hindi),


                              # Repeat the iterator for multiple epochs.
                              repeat=False, 

                              # Sort all examples in data using `sort_key`.
                              sort=False, 

                              # Shuffle data on each epoch run.
                              shuffle=True,

                              # Use `sort_key` to sort examples in each batch.
                              sort_within_batch = True,
                              
                              )

In [ ]:
for i in val_dataloader:
  print(i.Hindi.device)
  print(i.roman.T)
    

  break

cuda:0
torch.Size([32, 14])


In [ ]:
for i in [19, 14, 26, 15,  4, 17,  2, 16, 11]:
  print(hi_text.vocab.itos[i],end="")
print("\t")
for i in [ 2, 13, 11, 10,  7, 20, 18,  4,  4, 19,  6,  3,  1,  1]:
  print(en_text.vocab.itos[i],end="")

देशव्यापी	
<sos>s्रikdaaकn<eos><pad><pad>

1244

In [1]:
class Encoder(nn.Module):
    """ Encoder encodes the input sequence into a dense embedded context.
    """
    
    def __init__(self, input_size, embed_size, hidden_size, n_layers=1, dropout_rate=0.5):
        super(Encoder, self).__init__()
        self.input_size = input_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout_rate = 0.7 #dropout_rate
        self.is_cuda = True if torch.cuda.is_available() else False
        
        self.embedding = nn.Embedding(input_size, embed_size)
        self.rnn = nn.GRU(embed_size, hidden_size, n_layers, bidirectional=True, 
                          dropout=0.7,batch_first=False)
        self.rnn2 = nn.GRU(hidden_size*2,hidden_size,n_layers,dropout=0.4)
        self.fc = nn.Linear(3*hidden_size , hidden_size)
        self.dropout = nn.Dropout(self.dropout_rate)
    

    def load_pretrained_embeddings(self, embeddings, fine_tune=False):
        self.embedding.weight = nn.Parameter(embeddings)
        for p in self.embedding.parameters():
            p.requires_grad = fine_tune
    
    
    def forward(self, x, lengths):
        """ x: (seq_len, batch_size)
            lengths: (batch_size)
        """
        #lengths = torch.LongTensor(list(map(self.length, x.T)))
        
        lengths, perm_idx = lengths.sort(0, descending=True)
        
        x = x.T[perm_idx]
        x = x.T
        
        embed = self.dropout(self.embedding(x))  # (seq_len, batch_size, embed_size)
        
        

        # packing the embedding sequence to avoid unnecessary computations
        
        packed_embed = pack_padded_sequence(embed, lengths.cpu().numpy(),batch_first=False)
        #packed_embed = self.embedAndPack(x,batch_first=True)
        packed_outputs, hidden = self.rnn(packed_embed)
        
        outputs, _ = pad_packed_sequence(packed_outputs,batch_first=False)
        #outputs = self.dropout(outputs)  #new!
        packed_embed = pack_padded_sequence(outputs, lengths.cpu().numpy(),batch_first=False)
        packed_outputs ,hidden2 = self.rnn2(packed_outputs)
        outputs, _ = pad_packed_sequence(packed_outputs,batch_first=False)
        
        # outputs is a non-packed sequence of all hidden states from last layer
        # outputs: (seq_len, batch_size, hidden_size * 2)
        # hidden is the final non-padded time stepped state
        # hidden: (n_layers * 2, batch_size, hidden_size)
        # hidden2: (n_layers, batch_size, hidden_size)
        # initial decoder hidden state
        #hidden = F.relu(self.fc(torch.cat([h for h in hidden[-2:]], dim=1)))  # (batch_size, hidden_size)
        h_fwd= hidden[-2]
        h_back = hidden[-1]
        
        hidden = F.relu(self.fc(torch.cat((h_fwd,h_back,hidden2[-1]),dim=1))) #!new
      
        # outputs: (seq_len, batch_size, hidden_size * 2)
        # hidden: (batch_size, hidden_size)
        return outputs, hidden
      
  
    
    

NameError: ignored

In [15]:
cls = Encoder(input_size=INPUT_SIZE, embed_size=ENC_EMBED_SIZE, hidden_size =HIDDEN_SIZE, n_layers=N_LAYERS, dropout_rate=DROPOUT_RATE )
dls = DecoderWithAttention(output_size =OUTPUT_SIZE , embed_size = DEC_EMBED_SIZE, hidden_size=HIDDEN_SIZE, n_layers=N_LAYERS, dropout_rate=DROPOUT_RATE )
em = Seq2SeqWithAttention(cls,dls,pad_idx=PAD_IDX)
#em.cuda()
for i in td:
  x = em(i.Hindi,i.roman)
  print(x.shape)
  #print(h.shape)
  break

NameError: ignored

In [16]:
class BahdanauAttention(nn.Module):
    """ Implement Additive attention proposed by Bahdanau et al., 2014.
        
    """
    
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.hidden_size = hidden_size
        
        self.W1 = nn.Linear(hidden_size , hidden_size, bias=False)
        self.W2 = nn.Linear(hidden_size, hidden_size, bias=False)
        self.V = nn.Linear(hidden_size, 1, bias=False)
        
    
    def forward(self, hidden, enc_outputs, enc_masks):
        """ hidden: (batch_size, hidden_size)
            enc_outputs: (seq_len, batch_size, hidden_size * 2)
            enc_masks: (batch_size, seq_len)
        """
        seq_len = enc_outputs.shape[0]
        
        if len(hidden.shape) ==2:
          hidden = hidden.unsqueeze(1).repeat(1, seq_len, 1)  # (batch_size, seq_len, hidden_size)
          #print(f"hidden1 : {hidden.shape}")
        else:
          #hidden = hidden[1].unsqueeze(1).repeat(1,seq_len,1)
          #hidden =torch.sum(hidden.permute(1,0,2),dim=1).repeat(1,seq_len,1)
          hidden= torch.sum(hidden,dim=0).unsqueeze(0)
          hidden= hidden.permute(1,0,2).repeat(1,seq_len,1)
          
        enc_outputs = enc_outputs.permute(1, 0, 2)  # (batch_size, seq_len, hidden_size * 2)
        
        # calculate alignment scores
        energy = F.relu(self.W1(enc_outputs) + self.W2(hidden))  # (batch_size, seq_len, hidden_size)
        scores = self.V(energy)  # (batch_size, seq_len, 1)
        scores = scores.squeeze(2)  # (batch_size, seq_len)
        
        # mask out invalid positions
        scores = scores.masked_fill(enc_masks == 0, float('-inf'))  # (batch_size, seq_len)
        
        # calculate the attention weights (prob) from alignment scores
        alphas = F.softmax(scores, dim=-1)  # (batch_size, seq_len)
        
        # calculate context vector
        context_vector = torch.bmm(alphas.unsqueeze(1) ,enc_outputs)  # (batch_size, 1, hidden_size * 2)
        
        # context_vector: (batch_size, 1, hidden_size * 2)

        return context_vector, alphas
        

In [34]:
class DecoderWithAttention(nn.Module):
    """ Decoder decodes the encoded dense embedding context into a sequence 
        with use of attention mechanism.
    """
    def __init__(self, output_size, embed_size, hidden_size, n_layers=1, dropout_rate=0.5):
        super(DecoderWithAttention, self).__init__()
        self.output_size = output_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.n_layers = 2
        self.dropout_rate = 0.5 #dropout_rate
        self.lin_hidden_size_1 = 1024
        self.lin_hidden_size_2 = 2048
        self.embedding = nn.Embedding(output_size, embed_size)
        self.rnn = nn.GRU(embed_size + (hidden_size), self.hidden_size, self.n_layers,
                          bidirectional=False, dropout=0.6)
        self.attention = BahdanauAttention(hidden_size)
        self.fc = nn.Linear(embed_size + hidden_size + hidden_size,  self.lin_hidden_size_1 )
        self.fc2 = nn.Linear(embed_size + hidden_size + hidden_size+self.lin_hidden_size_1,  self.lin_hidden_size_2)
        self.fc3 = nn.Linear(self.lin_hidden_size_2+self.lin_hidden_size_1+hidden_size,  output_size)
        self.fc4 = nn.Linear(output_size, output_size)
        self.dropout = nn.Dropout(self.dropout_rate)
        self.dropout2 = nn.Dropout(0.4)#self.dropout_rate)
        self.act = nn.Tanh()
        
    
    def load_pretrained_embeddings(self, embeddings, fine_tune=False):
        self.embedding.weight = nn.Parameter(embeddings)
        for p in self.embedding.parameters():
            p.requires_grad = fine_tune
    
    
    def forward(self, x, hidden, enc_outputs, enc_masks):
        """ x: (batch_size)
            hidden: (batch_size, hidden_size)
            enc_outputs: (seq_len, batch_size, hidden_size * 2)
            enc_masks: (batch_size, seq_len)
        """
        embed = self.dropout(self.embedding(x.unsqueeze(0)))  # (1, batch_size, embed_size)
        
        # calculate attention weights and context vector
        context_vector, alphas = self.attention(hidden, enc_outputs, enc_masks)
        
        # context_vector: (batch_size, 1, hidden_size * 2)
        # alphas: (batch_size, seq_len)
        
        context_vector = context_vector.permute(1, 0, 2)  # (1, batch_size, hidden_size * 2)
        
        # input to the rnn is a concatenation of embedding and weighted encoder states (context)
        rnn_input = torch.cat((embed, context_vector), dim=2)  # (1, batch_size, embed_size + hidden_size * 2)
        
        if len(hidden.shape) == 2:
          hidden = torch.broadcast_to(hidden,(self.n_layers,hidden.shape[0],hidden.shape[1])).contiguous()
        # update the rnn hidden state
        output, hidden = self.rnn(rnn_input, hidden)
        
        # output: (seq_len=1, batch_size, hidden_size)
        # hidden: (n_layers=2, batch_size, hidden_size)
        # seq_len and n_layers are always equal to 1 for this decoder
        
        assert (output == hidden[1]).all()
        
        embed = embed.squeeze(0)  # (batch_size, embed_size)
        output = output.squeeze(0)  # (batch_size, hidden_size)
        context_vector = context_vector.squeeze(0)  # (batch_size, hidden_size * 2)
        
        x1 = self.act(self.dropout(self.fc(torch.cat((embed, output, context_vector), dim=1))))  # (batch_size, output_size)
        x = self.act(self.dropout2(self.fc2(torch.cat((x1,embed,output,context_vector),dim=1))))
        x = self.act(self.dropout(self.fc3(torch.cat((x1,x,context_vector),dim=1))))
        
        prediction = self.fc4(x)
        
        
        hidden = hidden.squeeze(0)  # (batch_size, hidden_size)
        
        # prediction: (batch_size, output_size)
        # hidden: (batch_size, hidden_size)
        # alphas: (batch_size, seq_len)
        return prediction, hidden, alphas
    def length(self,vector):
      new_vec = []
      for vec in vector:
        if not vec==1:
            new_vec.append(vec)
      return len(new_vec)
    

In [28]:
import random
class Seq2SeqWithAttention(nn.Module):
    """ Seq2Seq wrapper for encoder and decoder.
    """
    
    def __init__(self, encoder, decoder, pad_idx):
        super(Seq2SeqWithAttention, self).__init__()
        self.pad_idx = pad_idx
        self.device = ("cuda" if torch.cuda.is_available() else 'cpu')
        self.encoder = encoder
        self.decoder = decoder
    
    
    def forward(self, source, target, teacher_force_ratio=0.5):
        """ source: (source_seq_len, batch_size)
            source_lengths: (batch_size)
            target: (target_seq_len, batch_size)
        """
        source_lengths =torch.LongTensor(list(map(self.length, source.T)))
        source_lengths = source_lengths.to(source.device)
        
        target_seq_len, batch_size = target.shape
        target_vocab_size = self.decoder.output_size
        
        # initialize tensor to store decoder outputs
        outputs = torch.zeros((target_seq_len, batch_size, target_vocab_size)).to(source.device)
        
        # outputs: (target_seq_len, batch_size, output_size)
        
        # encode the source sequence
        enc_outputs, hidden = self.encoder(source,source_lengths)
        
        # enc_outputs: (seq_len, batch_size, hidden_size * 2)
        # hidden: (batch_size, hidden_size)
        
        # create mask for source sequence
        enc_masks = (source != self.pad_idx).permute(1, 0)  # (batch_size, seq_len)
        
        # initial input to the decoder is the SOS_TOKEN
        x = target[0, :]  # (batch_size)
        
        for t in range(1, target_seq_len):
            # decode the encoded sequence for current time step
            output, hidden, _ = self.decoder(x, hidden, enc_outputs, enc_masks)
            
            # output: (batch_size, output_size)
            # hidden: (batch_size, hidden_size)
            
            # store the decoder outputs for current time step
            outputs[t] = output
            
            # get the highest predicted token (best word) for current time step
            top1 = output.argmax(dim=1)
            
            # teacher forcing ratio to control how many times 
            # we feed the actual next word as input to the decoder
            # teacher forcing ratio of 0.5 indicates that 50% of the time
            # the decoder sees the actual next word
            x = target[t] if random.random() < teacher_force_ratio else top1
        
        # outputs: (target_seq_len, batch_size, output_size)
        return outputs
    def length(self,vector):
        new_vec = []
        for vec in vector:
          if not vec==1:
              new_vec.append(vec)
        return len(new_vec)
    

In [39]:
INPUT_SIZE = len(hi_text.vocab)  # source vocab size
OUTPUT_SIZE = len(en_text.vocab)  # target vocab size
ENC_EMBED_SIZE = 150
DEC_EMBED_SIZE = 150
HIDDEN_SIZE = 256
N_LAYERS = 1
DROPOUT_RATE = 0.5
N_EPOCHS = 50
CLIP = 0.5
PAD_IDX = 1
UNK_IDX = 0
SOS_IDX = 2
EOS_IDX = 3
print(f" Output_size = {OUTPUT_SIZE}")

 Output_size = 30


In [ ]:
em.cuda()

Seq2SeqWithAttention(
  (encoder): Encoder(
    (embedding): Embedding(16311, 512)
    (rnn): GRU(512, 256, dropout=0.2, bidirectional=True)
    (fc): Linear(in_features=512, out_features=256, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (decoder): DecoderWithAttention(
    (embedding): Embedding(30, 512)
    (rnn): GRU(1024, 256, dropout=0.2)
    (attention): BahdanauAttention(
      (W1): Linear(in_features=512, out_features=256, bias=False)
      (W2): Linear(in_features=256, out_features=256, bias=False)
      (V): Linear(in_features=256, out_features=1, bias=False)
    )
    (fc): Linear(in_features=1280, out_features=30, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
)

In [ ]:
en_text.vocab.itos[3]

In [20]:
def convert_ids_to_text(ids, vocab, eos_idx, unk_idx):
    """ Converts token ids to text.
    """
    if ids.dim() == 1:
        output_tokens = []
        for token_id in ids:
            if token_id == eos_idx:
                break
            else:
                output_tokens.append(vocab.itos[token_id])
        return output_tokens
    
    elif ids.dim() == 2:
        return [convert_ids_to_text(ids[:, i], vocab, eos_idx, unk_idx) for i in range(ids.size(1))]
    
    raise RuntimeError(f'ids has {ids.size()} dimensions, expected 2 dimensions')

In [21]:
def train_fn(model, iterator, optimizer, criterion, clip=1.0):
    
    
    total_loss = 0
    steps = 0
    model.train()
    tk0 = tqdm(iterator, total=len(iterator), position=0, leave=True)
    
    for idx, batch in enumerate(tk0):
        source = batch.Hindi
        target = batch.roman

        # source: (source_seq_len, batch_size), source_lengths: (batch_size)
        # target: (target_seq_len, batch_size), target_lengths: (batch_size)

        # forward pass
        optimizer.zero_grad()
        output = model(source, target)  # (target_seq_len, batch_size, output_size)
        
        # ignoring SOS_TOKEN
        output = output[1:]  # ((target_seq_len - 1), batch_size, output_size)
        target = target[1:]  # ((target_seq_len - 1), batch_size)
        
        # calculate the loss
        loss = criterion(
            output.view(-1, output.shape[-1]),  # ((target_seq_len - 1) * batch_size, output_size)
            target.view(-1)  # ((target_seq_len - 1) * batch_size)
        )
        total_loss += loss.item()
        steps += 1

        output = output.argmax(dim=-1)  # ((target_seq_len - 1), batch_size)

        # backward pass
        loss.backward()

        # clip gradients to avoid exploding gradients issue
        nn.utils.clip_grad_norm_(model.parameters(), clip)

        # update model parameters
        optimizer.step()
        
        tk0.set_postfix(loss=total_loss/steps)
        
    tk0.close()

    perplexity = np.exp(total_loss / len(iterator))
    
    return output, perplexity
def eval_fn(model, iterator, criterion):
    model.eval()
    total_loss = 0.0
    steps = 0
    hypotheses = []
    references = []
    
    tk0 = tqdm(iterator, total=len(iterator), position=0, leave=True)
    
    with torch.no_grad():
        for idx, batch in enumerate(tk0):
            source = batch.Hindi
            target= batch.roman
            
            # source: (source_seq_len, batch_size), source_lengths: (batch_size)
            # target: (target_seq_len, batch_size), target_lengths: (batch_size)
            
            # forward pass
            output = model(source,target)  # (target_seq_len, batch_size, output_size)
            
            # ignoring SOS_TOKEN
            output = output[1:]  # ((target_seq_len - 1), batch_size, output_size)
            target = target[1:]  # ((target_seq_len - 1), batch_size)
            
            # calculate the loss
            loss = criterion(
                output.view(-1, output.shape[-1]),  # ((target_seq_len - 1) * batch_size, output_size)
                target.view(-1)  # ((target_seq_len - 1) * batch_size)
            )
            total_loss += loss.item()
            steps += 1
            
            output = output.argmax(dim=-1)  # ((target_seq_len - 1), batch_size)

            # converting the ids to tokens (used later for calculating BLEU score)
            pred_tokens = convert_ids_to_text(output, en_text.vocab, EOS_IDX, UNK_IDX)
            target_tokens = convert_ids_to_text(target, en_text.vocab, EOS_IDX, UNK_IDX)

            hypotheses += pred_tokens
            references += [[token] for token in target_tokens]
            
            tk0.set_postfix(loss=total_loss/steps)
            
    tk0.close()
            
    perplexity = np.exp(total_loss / len(iterator))
    bleu4 = bleu_score(hypotheses, references)

    return output, perplexity, bleu4

'cuda'

In [40]:
cls = Encoder(input_size=INPUT_SIZE, embed_size=ENC_EMBED_SIZE, hidden_size =HIDDEN_SIZE, n_layers=N_LAYERS, dropout_rate=DROPOUT_RATE )
dls = DecoderWithAttention(output_size =OUTPUT_SIZE , embed_size = DEC_EMBED_SIZE, hidden_size=HIDDEN_SIZE, n_layers=N_LAYERS, dropout_rate=DROPOUT_RATE )
model = Seq2SeqWithAttention(cls,dls,pad_idx=PAD_IDX)
if device == 'cuda':
  model.cuda()

In [41]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-2)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=3, threshold=0.001, mode='max'
)

In [24]:
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)


125

In [37]:
# for reproducibility
# refer https://pytorch.org/docs/stable/notes/randomness.html
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [42]:
best_bleu4 = float('-inf')
es_patience = 10
patience = 0
model_path = 'model(rms).pth'


for epoch in range(0, N_EPOCHS + 1):
    # one epoch training
    _, train_perplexity = train_fn(model, train_dataloader, optimizer, criterion, CLIP)
    
    # one epoch validation
    _, valid_perplexity, valid_bleu4 = eval_fn(model, val_dataloader, criterion)
    
    print(f'Epoch: {epoch}, Train perplexity: {train_perplexity:.4f}, Valid perplexity: {valid_perplexity:.4f}, Valid BLEU4: {valid_bleu4:.4f}')
    
    scheduler.step(valid_bleu4)
    
    # early stopping
    is_best = valid_bleu4 > best_bleu4
    if is_best:
        print(f'BLEU score improved ({best_bleu4:.4f} -> {valid_bleu4:.4f}). Saving Model!')
        best_bleu4 = valid_bleu4
        patience = 0
        torch.save(model.state_dict(), model_path)
    else:
        patience += 1
        print(f'Early stopping counter: {patience} out of {es_patience}')
        if patience == es_patience:
            print(f'Early stopping! Best BLEU4: {best_bleu4:.4f}')
            break
    model.eval()
    for i in val_dataloader:
        input = i.Hindi
        target = i.roman
        output = model(input,target)
        output = output[1:]  # ((target_seq_len - 1), batch_size, output_size)
        target = target[1:]
  
        output = output.argmax(dim=-1) 
        pred_words = convert_ids_to_text(output, en_text.vocab, EOS_IDX, UNK_IDX)
        actual = convert_ids_to_text(target, en_text.vocab, EOS_IDX, UNK_IDX)
        for a,c in zip(actual,pred_words):
          word = ""
          pred = ""
          for b in a:
            word +=b
          for d in c:
            pred +=d
          print(f"{word}              {pred}")
          break
        break
  

  0%|          | 0/78 [00:00<?, ?it/s]

  0%|          | 0/139 [00:00<?, ?it/s]

Epoch: 0, Train perplexity: 18.0604, Valid perplexity: 19.0696, Valid BLEU4: 0.0000
BLEU score improved (-inf -> 0.0000). Saving Model!
loklubhavan              aaaaaaaaaaaaa


  0%|          | 0/78 [00:00<?, ?it/s]

  0%|          | 0/139 [00:00<?, ?it/s]

...

  0%|          | 0/139 [00:00<?, ?it/s]

Epoch: 9, Train perplexity: 17.3014, Valid perplexity: 17.2910, Valid BLEU4: 0.0000
Early stopping counter: 9 out of 10
seby              aaaaaaaaaaaaa


  0%|          | 0/78 [00:00<?, ?it/s]

  0%|          | 0/139 [00:00<?, ?it/s]

Epoch: 10, Train perplexity: 17.2948, Valid perplexity: 17.3009, Valid BLEU4: 0.0000
Early stopping counter: 10 out of 10
Early stopping! Best BLEU4: 0.0000


In [ ]:
model.eval()
for i in val_dataloader:
  input = i.Hindi
  target = i.roman
  output = model(input,target)
  output = output[1:]  # ((target_seq_len - 1), batch_size, output_size)
  target = target[1:]
  print(output.shape)
  output = output.argmax(dim=-1) 
  pred_words = convert_ids_to_text(output, en_text.vocab, EOS_IDX, UNK_IDX)
  actual = convert_ids_to_text(target, en_text.vocab, EOS_IDX, UNK_IDX)
  print(output)
  for a,c in zip(actual,pred_words):
    word = ""
    pred = ""
    for b in a:
       word +=b
    for d in c:
      pred +=d
    print(len(pred))
    print(f"{pred}              {pred}")
    
  break
  

torch.Size([12, 32, 30])
tensor([[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3],
        [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4],
        [3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 4, 4, 4],
        [4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 3, 3, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 3, 3, 3],
        [3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 4, 4, 4],
        [4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 3, 3, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 3, 3, 3],
        [3, 4, 4, 4, 3, 3, 3, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 3, 4, 3, 3, 3, 3, 4,
         3, 4, 4, 3, 3, 4, 4, 3],
        [4, 3, 3, 3, 4, 4, 4, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 4, 3, 4, 4, 4, 4, 3,
         4, 3, 3, 4, 4, 3, 3, 4],
        [4, 3, 3, 3, 4, 4, 3, 4, 4, 4, 3, 4, 3, 4, 4, 3

In [ ]:
for a,c in zip(actual,pred_words):
  word = ""
  pred = ""
  for b in a:
     word +=b
  for d in c:
    pred +=d
  print(f"{word}              {pred}")

  

riboflavin              aaatiartion
taakta              aaaaaa
inhaler              aaaaaan
rahaseyavaade              aaaaahaan
richi              aaaa
gher              aaa
jhaptane              aaaaaa
yogasanon              aaaaahaa
restaran              aaaaaaaa
gramon              aaaaan
paayal              aaaa
jayram              aaaaa
loli              aaaa
vaad              aaa
ringtone              aaraaaa
zaidi              aaaa
paathak              aaaa
rhasywad              aaraaana
dikha              aaaaaa
aladin              aaaaaya
tex              aaaaaanan
khelte              aaaaaa
satven              aaaaan
shroton              aaaaaaa
bharosaa              aaaaan
lautne              aaaaa
mirgi              aaaaaya
paanchvin              aaaaahaa
paarvan              aaaaaan
shergarh              aaaaaa
districts              aaraaaaharyaa
cpr              aaaaaan


In [ ]:
def greedy_decode(model, sentence, max_len=100):
    model.eval()
    
    if isinstance(sentence, str):
        tokens = [token.text.lower() for token in spacy_de(sentence)]
    else:
        tokens = [token.lower() for token in sentence]
    
    token_ids = [SOS_IDX] + [en_text.vocab.stoi.get(token, UNK_TOKEN) for token in tokens] + [EOS_IDX]
    
    source = torch.tensor(token_ids, dtype=torch.long).unsqueeze(1).to(device)  # (source_seq_len, 1)
    source_length = torch.tensor([len(source)], dtype=torch.long)  # (source_seq_len)
    
    # encode the source sequence
    with torch.no_grad():
        enc_outputs, hidden = model.encoder(source, source_length)
    
    # enc_outputs: (source_seq_len, 1, hidden_size * 2)
    # hidden: (1, hidden_size)
    
    # create mask for source sequence
    enc_masks = (source != PAD_IDX).permute(1, 0)  # (1, seq_len)
    
    target_ids = [SOS_IDX]
    attention_scores = torch.zeros(max_len, 1, len(token_ids)).to(device)  # (max_len, 1, source_seq_len)
    
    for t in range(max_len):
        target = torch.tensor([target_ids[-1]], dtype=torch.long).to(device)
        
        # decode the sequence
        with torch.no_grad():
            output, hidden, attention = model.decoder(target, hidden, enc_outputs, enc_masks)
            
            # output: (1, output_size)
            # hidden: (1, hidden_size)
            # attention: (1, source_seq_len)
            
        attention_scores[t] = attention
        
        target_id = output.argmax(dim=-1).item()
        target_ids.append(target_id)
        
        # stop decoding if we encounter EOS_TOKEN or reach the max length
        if target_id == EOS_IDX or len(target_ids) >= max_len:
            break
    
    target_tokens = [de_text.vocab.itos[id] for id in target_ids]
    attention_scores = attention_scores[:len(target_tokens)-1].squeeze(1).cpu().detach().numpy()

    # target_tokens: list of size (target_seq_len - 1)
    # attention_scores: (target_seq_len - 1, 1, source_seq_len)
    
    return target_tokens[1:], attention_scores

In [ ]:
for i in range(2304,2416):
  print(chr(i),end=" ")

ऀ ँ ं ः ऄ अ आ इ ई उ ऊ ऋ ऌ ऍ ऎ ए ऐ ऑ ऒ ओ औ क ख ग घ ङ च छ ज झ ञ ट ठ ड ढ ण त थ द ध न ऩ प फ ब भ म य र ऱ ल ळ ऴ व श ष स ह ऺ ऻ ़ ऽ ा ि ी ु ू ृ ॄ ॅ ॆ े ै ॉ ॊ ो ौ ् ॎ ॏ ॐ ॑ ॒ ॓ ॔ ॕ ॖ ॗ क़ ख़ ग़ ज़ ड़ ढ़ फ़ य़ ॠ ॡ ॢ ॣ । ॥ ० १ २ ३ ४ ५ ६ ७ ८ ९ 

In [ ]:

"""
Since character embeddings are a bit weak in pytorch 3, this will hopefully help out
I think these should be trainable and also, invertable!
So you can actually recover output from the embeddings using Cos Similarity
"""

class CharacterEmbedding:
    def __init__(self, embedding_size,vocab):

        self.vocab = vocab
        self.embed = nn.Embedding(len(self.vocab), embedding_size)
        self.is_cuda = False
        self.cos = nn.CosineSimilarity(dim=2)

    def flatten(self, l):
        return list(itertools.chain.from_iterable(l))
    
    
    def length(self,vector):
      new_vec = []
      for vec in vector:
        if not vec==1:
            new_vec.append(vec)
      return len(new_vec)

    def embedAndPack(self, vectorized_seqs, batch_first=False):

        

        # get the length of each seq in your batch
        seq_lengths = torch.LongTensor(list(map(self.length, vectorized_seqs)))
        seq_lengths = seq_lengths.cuda() if self.is_cuda else seq_lengths
        print("seq len ",seq_lengths)
        # dump padding everywhere, and place seqs on the left.
        # NOTE: you only need a tensor as big as your longest sequence
        seq_tensor = Variable(torch.zeros((len(vectorized_seqs), seq_lengths.max()))).long()
        print(seq_tensor.shape)
        seq_tensor = seq_tensor.cuda() if self.is_cuda else seq_tensor
        
        for idx, (seq, seqlen) in enumerate(zip(vectorized_seqs, seq_lengths)):
            seq_tensor[idx, :seqlen] = torch.LongTensor(seq[:seqlen]).cuda() if self.is_cuda else torch.LongTensor(seq[:seqlen])
        print(seq_tensor)
        # SORT YOUR TENSORS BY LENGTH!
        seq_lengths, perm_idx = seq_lengths.sort(0, descending=True)
        print(seq_lengths)
        seq_tensor = seq_tensor[perm_idx]

        # utils.rnn lets you give (B,L,D) tensors where B is the batch size, L is the maxlength, if you use batch_first=True
        # Otherwise, give (L,B,D) tensors
        if not batch_first:
            seq_tensor = seq_tensor.transpose(0,1) # (B,L,D) -> (L,B,D)

        # embed your sequences
        seq_tensor = self.embed(seq_tensor)
        print(seq_tensor.shape)
        # pack them up nicely
        return pack_padded_sequence(seq_tensor, seq_lengths.cpu().numpy(), batch_first=True)

    def cuda(self):
        self.is_cuda = True
        self.embed = self.embed.cuda()
        return self

    def unpackToSequence(self, packed_output,to_words=False):
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        words = self.unembed(output)
        return words

    def unembed(self, embedded_sequence):
        weights = self.embed.state_dict()['weight']
        weights = weights.transpose(0,1).unsqueeze(0).unsqueeze(0)
        e_sequence = embedded_sequence.unsqueeze(3).data
        cosines = self.cos(e_sequence, weights)
        _, indexes = torch.topk(cosines, 1, dim=2)

        words = []
        for word in indexes:
            word_l = ''
            for char_index in word:
                word_l += self.vocab.itos[char_index[0]]
            words.append(word_l)
        return words









if __name__ == '__main__':

    seqs = i.Hindi.T
    print(seqs.shape)
    # make model

    embedding = CharacterEmbedding(embedding_size=5,vocab=hi_text.vocab)

    lstm = nn.LSTM(5, 5, batch_first=True)

    packed_input = embedding.embedAndPack(seqs, batch_first=True)
    
    words = embedding.unpackToSequence(packed_input)
    
    print(words)

    # throw them through your LSTM (remember to give batch_first=True here if you packed with it)
    packed_output, (ht, ct) = lstm(packed_input)

    words = embedding.unpackToSequence(packed_output)


    print(words)


torch.Size([32, 10])
seq len  tensor([ 4,  7,  7,  9,  3,  6,  4,  5,  9,  5,  7,  5,  5,  5,  5,  6, 10,  6,
         5,  6,  3,  5,  6,  8,  4, 10,  5,  4,  8,  6,  4,  4])
torch.Size([32, 10])
tensor([[39, 15,  2,  8,  0,  0,  0,  0,  0,  0],
        [25,  2,  3,  4, 13, 18,  6,  0,  0,  0],
        [ 8, 23, 41,  8,  2, 41, 27,  0,  0,  0],
        [ 8,  4, 22,  4,  3,  5, 16,  4,  8,  0],
        [ 9,  2, 16,  0,  0,  0,  0,  0,  0,  0],
        [16, 36, 10,  9, 11,  8,  0,  0,  0,  0],
        [31,  2, 24, 11,  0,  0,  0,  0,  0,  0],
        [ 8,  3, 16, 10, 29,  0,  0,  0,  0,  0],
        [ 8, 13,  8,  4, 17,  2, 39, 14, 10,  0],
        [39, 20,  4, 54,  2,  0,  0,  0,  0,  0],
        [17,  2,  9,  2, 17,  2,  9,  0,  0,  0],
        [ 8, 18, 29,  7,  3,  0,  0,  0,  0,  0],
        [25, 23,  8, 36,  6,  0,  0,  0,  0,  0],
        [37,  6, 39, 44, 37,  0,  0,  0,  0,  0],
        [37,  7,  2, 19, 26,  0,  0,  0,  0,  0],
        [46,  5, 16,  7, 12, 11,  0,  0,  0,  0],
    